# 한빛 아카데미 
url = 'https://www.genie.co.kr/chart/top200' <br/>
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [ ]:
base_url = 'https://www.hanbit.co.kr'
sub_url =  '/academy/books/new_book_list.html'
url = base_url + sub_url + '?page=' + str(1)
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

soup

<!DOCTYPE html>

<html lang="ko">
<head>
<!--[if lte IE 8]>
<script>
  location.replace('/support/explorer_upgrade.html');
</script>
<![endif]-->
<meta charset="utf-8"/>
<title>한빛아카데미</title>
<link href="https://www.hanbit.co.kr/images/common/hanbit.ico" rel="shortcut icon"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="website" property="og:type"/>
<meta content="한빛아카데미" property="og:title"/>
<meta content="이공계 분야 종합 교재 출판사로, 독자의 신뢰를 최우선 원칙으로 삼고 가르치는 교수님들과 배우는 학생들에게 든든한 조력자가 되도록 노력하고 있습니다." property="og:description"/>
<meta content="https://www.hanbit.co.kr/images/hanbitpubnet_logo.jpg" property="og:image"/>
<meta content="https://www.hanbit.co.kr/academy/books/new_book_list.html?page=1" property="og:url"/>
<link href="https://www.hanbit.co.kr/academy/books/new_book_list.html?page=1" rel="canonical">
<meta content="한빛아카데미,책,전자책,ebook,대학교재,교재출판사,강의교안,교수회원,자격증,정보교과서" name="keywords"/>
<meta content="이공계 분야 종합 교재 출판사로, 독자의 신뢰를 최우선 원칙으로 삼고 가르치는 교수님들과 배우는 학생들에게 든든한

### 1.찾으려고 하는 목록 데이터

In [ ]:
lis = soup.select('li.sub_book_list')


'/academy/books/book_view.html?p_code=B8672666879'

### 2. 하나의 데이터로 원하는 정보 추출

In [ ]:

li = lis[0]
href = li.find('a')['href']
href

'/academy/books/book_view.html?p_code=B8672666879'

### sub page로 이동해서 원하는 정보 추출

In [ ]:
sub_req = requests.get(base_url + href)
sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

In [ ]:
title = sub_soup.select_one(".store_product_info_box > h3").get_text().strip()
title

'STEM CookBook, 문제 해결력을 키우는 기계설계'

In [ ]:
info_lis = sub_soup.select('.info_list > li')
info_dict = {}
for li in info_lis:
  key = li.find('strong').get_text().split(' ')[0]
  val = li.find('span').get_text().strip()
  info_dict[key] = val

info_dict  

{'ISBN': '9791156646150',
 '물류코드': '4615',
 '저자': '이상범',
 '출간': '2022-08-05',
 '페이지': '680 쪽'}

In [ ]:
author = info_dict['저자']
try:
  translator = info_dict['번역']
except:
  translator = '-'
date = info_dict['출간']
page = int(info_dict['페이지'].split()[0].replace(',', ''))
author, translator, date, page

('이상범', '-', '2022-08-05', 680)

In [ ]:
price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',', ''))
price

34000

한 페이지내의 모든 정보 가져오기

In [24]:
lines = []
for li in lis:
  href = li.find('a')['href']
  sub_req = requests.get(base_url + href)
  sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

  title = sub_soup.select_one(".store_product_info_box > h3").get_text().strip()
  info_lis = sub_soup.select('.info_list > li')
  info_dict = {}
  for li in info_lis:
    key = li.find('strong').get_text().split(' ')[0]
    val = li.find('span').get_text().strip()
    info_dict[key] = val
  author = info_dict['저자']
  try:
    translator = info_dict['번역']
  except:
    translator = '-'
  date = info_dict['출간']
  page = int(info_dict['페이지'].split()[0].replace(',', ''))
  price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',', ''))
  lines.append([title, translator, date, page, price])

df = pd.DataFrame(lines, columns=['제목', '번역', '날짜', '페이지', '가격'])
df

,제목,번역,날짜,페이지,가격
0,"STEM CookBook, 문제 해결력을 키우는 기계설계",-,2022-08-05,680,34000
1,경영분석(6판),-,2022-07-22,556,36000
2,"IT CookBook, 기초부터 시작하는 PLC : OMRON NX",-,2022-07-18,636,32000
3,"IT CookBook, 원리로 이해하는 전력전자공학(2판)",-,2022-07-11,588,30000
4,"IT CookBook, TCP/IP 소켓 프로그래밍(2판)",-,2022-06-30,600,32000
5,"IT CookBook, 쉽게 배우는 데이터 통신과 컴퓨터 네트워크(3판)",-,2022-06-30,676,30000
6,"IT CookBook, 우분투 리눅스(3판)",-,2022-06-30,800,34000
7,거시경제학(3판),-,2022-06-30,640,36000
8,실용적 접근 방식의 경영학원론(10판),김안드레아,2022-06-30,800,39000
9,"IT CookBook, C로 시작하는 컴퓨터 프로그래밍(4판)",-,2022-06-26,432,23000


### 5. 10 페이지 정보 가져오기

In [29]:
lines = []
for page in tqdm(range(1, 11)):
  url = base_url + sub_url + '?page=' + str(page)
  req = requests.get(url)
  soup = BeautifulSoup(req.text, 'html.parser')
  lis = soup.select('li.sub_book_list')
  for li in lis:
    href = li.find('a')['href']
    sub_req = requests.get(base_url + href)
    sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

    title = sub_soup.select_one(".store_product_info_box > h3").get_text().strip()
    info_lis = sub_soup.select('.info_list > li')
    info_dict = {}
    for li in info_lis:
      key = li.find('strong').get_text().split(' ')[0]
      val = li.find('span').get_text().strip()
      info_dict[key] = val
    author = info_dict['저자']
    try:
      translator = info_dict['번역']
    except:
      translator = '-'
    date = info_dict['출간']
    page = int(info_dict['페이지'].split()[0].replace(',', ''))
    price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',', ''))
    lines.append([title, translator, date, page, price])

  0%|          | 0/10 [00:00<?, ?it/s]

In [31]:

df = pd.DataFrame(lines, columns=['제목', '번역', '날짜', '페이지', '가격'])

df

,제목,번역,날짜,페이지,가격
0,"STEM CookBook, 문제 해결력을 키우는 기계설계",-,2022-08-05,680,34000
1,경영분석(6판),-,2022-07-22,556,36000
2,"IT CookBook, 기초부터 시작하는 PLC : OMRON NX",-,2022-07-18,636,32000
3,"IT CookBook, 원리로 이해하는 전력전자공학(2판)",-,2022-07-11,588,30000
4,"IT CookBook, TCP/IP 소켓 프로그래밍(2판)",-,2022-06-30,600,32000
...,...,...,...,...,...
195,"IT Cookbook, 처음 만나는 전자회로",-,2019-07-10,440,24000
196,스트랭 미분방정식과 선형대수학,한빛수학교재연구소,2019-07-09,552,30000
197,"IT CookBook, 네트워크 개론(3판)",-,2019-07-05,516,25000
198,"IT CookBook, 텐서플로로 배우는 딥러닝",-,2019-06-30,448,27000


In [32]:
df.to_csv('한빛신간도서(220808).csv', index=False)